# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [13]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## 1. Select stocks from the Dow 30

In [14]:
import requests
from bs4 import BeautifulSoup
import re
import json

# Step 1: Get the webpage content
url = 'https://www.cnbc.com/dow-30/'  # Example CNBC page
response = requests.get(url)
webpage_content = response.text

# Step 2: Parse HTML using BeautifulSoup
soup = BeautifulSoup(webpage_content, 'html.parser')

# Step 3: Use regex to find the JavaScript object
pattern = r'window\.__s_data\s*=\s*(\{.*?\});'
script_tag = soup.find("script", text=re.compile(pattern))

if script_tag:
    # Step 4: Extract the JavaScript object
    match = re.search(pattern, script_tag.string)
    if match:
        js_object = match.group(1)
        
        # Step 5: Convert JavaScript object to Python dictionary
        js_object = js_object.replace("null", "None").replace("false", "False").replace("true", "True")
        data = eval(js_object)
        
        # Step 6: Navigate to the ticker symbols within the data structure
        ticker_symbols = []
        try:
            # Digging into the 'marketsSectionTable' part to find ticker symbols
            market_table = data['page']['page']['layout']
            for layout in market_table:
                for column in layout['columns']:
                    for module in column['modules']:
                        if module['name'] == 'marketsSectionTable':
                            for chart in module['data']['relatedMarketCharts']:
                                for issue in chart['chartIssueDetail']:
                                    ticker_symbols.append(issue['symbol'])
        
            # Step 7: Print the extracted ticker symbols
            print("Extracted Ticker Symbols:")
            print(ticker_symbols)
        except KeyError as e:
            print(f"Key not found: {e}")
    else:
        print("JavaScript object not found in script.")
else:
    print("Script tag not found.")

Extracted Ticker Symbols:
['AMZN', 'AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WMT', 'DIS', 'DOW']


C:\Users\Brian Ngetich\AppData\Local\Temp\ipykernel_4812\3376695251.py:16: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  script_tag = soup.find("script", text=re.compile(pattern))


In [20]:
# Adding a column based on a condition
stocks['is_dow_30'] = stocks['Ticker'].apply(lambda x: 1 if x in ticker_symbols else 0 )

# Selecting only the stocks in Dow 30 because only the info from the MarketStacks
select_stocks = stocks[stocks["is_dow_30"] == 1]

print(select_stocks)

    Ticker  is_dow_30
3     AAPL          1
33    AMGN          1
36    AMZN          1
53     AXP          1
55      BA          1
79     CAT          1
114    CRM          1
115   CSCO          1
123    CVX          1
134    DIS          1
141    DOW          1
210     GS          1
217     HD          1
224    HON          1
234    IBM          1
242   INTC          1
258    JNJ          1
260    JPM          1
270     KO          1
298    MCD          1
311    MMM          1
316    MRK          1
320   MSFT          1
334    NKE          1
367     PG          1
443    TRV          1
458    UNH          1
464      V          1
475     VZ          1
487    WMT          1


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets_py import MARKETSTACK_API_TOKEN
from secrets_py import TIINGO_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
headers = {
    'Content-Type': 'application/json'
}

symbol = "AAPL"
market_cap_url = f"https://api.tiingo.com/tiingo/fundamentals/{symbol}/daily?token={TIINGO_API_TOKEN}&startDate=2024-10-10"
price_url = f"https://api.tiingo.com/tiingo/daily/{symbol}/prices?token={TIINGO_API_TOKEN}&startDate=2024-10-10"
data_1 = requests.get(market_cap_url, headers=headers)
data_2 = requests.get(price_url, headers=headers)
print(data_2.json())

[{'date': '2024-10-10T00:00:00.000Z', 'close': 229.04, 'high': 229.5, 'low': 227.17, 'open': 227.78, 'volume': 28183544, 'adjClose': 229.04, 'adjHigh': 229.5, 'adjLow': 227.17, 'adjOpen': 227.78, 'adjVolume': 28183544, 'divCash': 0.0, 'splitFactor': 1.0}, {'date': '2024-10-11T00:00:00.000Z', 'close': 227.55, 'high': 229.41, 'low': 227.34, 'open': 229.3, 'volume': 31759188, 'adjClose': 227.55, 'adjHigh': 229.41, 'adjLow': 227.34, 'adjOpen': 229.3, 'adjVolume': 31759188, 'divCash': 0.0, 'splitFactor': 1.0}, {'date': '2024-10-14T00:00:00.000Z', 'close': 231.3, 'high': 231.73, 'low': 228.6, 'open': 228.7, 'volume': 39882085, 'adjClose': 231.3, 'adjHigh': 231.73, 'adjLow': 228.6, 'adjOpen': 228.7, 'adjVolume': 39882085, 'divCash': 0.0, 'splitFactor': 1.0}, {'date': '2024-10-15T00:00:00.000Z', 'close': 233.85, 'high': 237.49, 'low': 232.37, 'open': 233.61, 'volume': 64751367, 'adjClose': 233.85, 'adjHigh': 237.49, 'adjLow': 232.37, 'adjOpen': 233.61, 'adjVolume': 64751367, 'divCash': 0.0, 's

In [5]:
data_1.json()

[{'date': '2024-10-10T00:00:00.000Z',
  'marketCap': 3512118141280.0,
  'enterpriseVal': 3551621141280.0,
  'peRatio': 34.4473904555,
  'pbRatio': 52.6491296588,
  'trailingPEG1Y': 3.3652450676},
 {'date': '2024-10-11T00:00:00.000Z',
  'marketCap': 3489270359100.0,
  'enterpriseVal': 3528773359100.0,
  'peRatio': 34.2232959228,
  'pbRatio': 52.3066252788,
  'trailingPEG1Y': 3.3433527555},
 {'date': '2024-10-14T00:00:00.000Z',
  'marketCap': 3546773166600.0,
  'enterpriseVal': 3586276166600.0,
  'peRatio': 34.78729223,
  'pbRatio': 53.1686329466,
  'trailingPEG1Y': 3.3984508563},
 {'date': '2024-10-15T00:00:00.000Z',
  'marketCap': 3585875075700.0,
  'enterpriseVal': 3625378075700.0,
  'peRatio': 35.1708097189,
  'pbRatio': 53.7547981606,
  'trailingPEG1Y': 3.4359175648},
 {'date': '2024-10-16T00:00:00.000Z',
  'marketCap': 3554133525960.0,
  'enterpriseVal': 3593636525960.0,
  'peRatio': 34.8594837573,
  'pbRatio': 53.278969928,
  'trailingPEG1Y': 3.4055034132},
 {'date': '2024-10-17T0

In [6]:
price_list = data_2.json()
price_dict = price_list[0]
marcap_list = data_1.json()
marcap_dict = marcap_list[0]

latest_price = price_dict['close']
market_cap = marcap_dict['marketCap']

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [7]:
# Define your columns
my_columns = ["Ticker", "Stock Price", "Market Capitalization", "Number of Shares to Buy"]

# Create an empty DataFrame with specified columns
final_dataframe = pd.DataFrame(columns=my_columns)

other_column = "N/A"
# Add a new row using .loc[] at the next available index
final_dataframe.loc[len(final_dataframe)] = [symbol, latest_price, market_cap, other_column]

print(final_dataframe)

  Ticker  Stock Price  Market Capitalization Number of Shares to Buy
0   AAPL       229.04           3.512118e+12                     N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [10]:
marcap_list = data_1.json()
marcap_list

{'detail': 'Error: Free and Power plans are limited to the DOW 30. If you would like access to all supported tickers, then please E-mail support@tiingo.com to get the Fundamental Data API added as an add-on service.'}

In [8]:
final_dataframe = pd.DataFrame(columns=my_columns)

select_stocks = stocks.filter(is_dow_30 = 1)

for stock in select_stocks["Ticker"][:5]:
  price_url = f"https://api.tiingo.com/tiingo/daily/{stock}/prices?token={TIINGO_API_TOKEN}&startDate=2024-10-10"
  response_1 = requests.get(price_url, headers=headers)
  if response_1.status_code == 200:
    price_list = data_2.json()
    price_dict = price_list[0]
  
  latest_price = price_dict['close']
  
  final_dataframe.loc[len(final_dataframe)] = [stock, latest_price, market_cap, other_column]

final_dataframe

KeyError: 0

In [ ]:
final_dataframe = pd.DataFrame(columns=my_columns)

select_stocks = stocks.filter(is_dow_30 = 1)

for stock in select_stocks["Ticker"][:5]:
  market_cap_url = f"https://api.tiingo.com/tiingo/fundamentals/{stock}/daily?token={TIINGO_API_TOKEN}&startDate=2024-10-10"
  response_2 = requests.get(market_cap_url, headers=headers)
  if response_2.status_code == 200:
    marcap_list = data_1.json()
    marcap_dict = marcap_list[0]

  market_cap = marcap_dict['marketCap']
  
  final_dataframe.loc[len(final_dataframe)] = [stock, latest_price, market_cap, other_column]

final_dataframe


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: